## **Glassdoor data analysis and modeling**

### Part 2: __Data cleaning__


In [1]:
#### Parsing Cols: 
    # Salary Estimate 
    # Company name 
    # State name 
    # Age of company  
    # Job description 

In [69]:
import pandas as pd 
import numpy as np
# Display full output 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.__version__

'0.23.4'

In [70]:
df = pd.read_csv("dataset/gd_data.csv")


In [71]:
print("Data having Salary Estimate:")
df.shape
print("Data not having Salary Estimate:")
df[df["Salary Estimate"] == "-1"].shape 

# Separate data with and without "Salary estimate"
# Save data not having Salary Estimate to further infer it in later 
df_no_salary = df[df["Salary Estimate"] == "-1"]
df.to_csv('dataset/gd_data_no_salary_estimate.csv', index=False)

Data having Salary Estimate:


(956, 15)

Data not having Salary Estimate:


(214, 15)

In [72]:
# SALARY PARSING 
#  Filter out only data having "Salary Estimate" 
df = df[df["Salary Estimate"] != "-1"]
# Remove string after "Salary Estimate" - Only take the salary range 
salary = df["Salary Estimate"].apply(lambda x: x.split('(')[0])
# Remove Thousand & Dollar signs  
salary = salary.apply(lambda x: x.replace('K','').replace('$',''))
# Remove unecessary string surrounding the "Salary Estimate" 
salary = salary.apply(lambda x: x.lower().replace('employer provided salary:','').replace('per hour',''))

# As shown above, salary may come "Per hour" or "Employer Provided" or simply "Per annum"
# Create new Features for handling so 
# 1: if 'Salary Estimate' is hourly, 0: otherwise 
# 1: if 'Salary Estimate' is employer provided, 0: otherwise
df["hourly"] = salary.apply(lambda x: 1 if 'per hour' in x.lower() else 0)
df["employer_provided"] = salary.apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)
#df["employer_provided"].unique()


# Split and get the min. and max. salary estimate and make them as new features  
df["min_salary"] = salary.apply(lambda x: int(x.split('-')[0]))
df["max_salary"] = salary.apply(lambda x: int(x.split('-')[1]))
df["avg_salary"] = (df["min_salary"] + df["max_salary"]) / 2
# salary.unique()
# df["avg_salary"].dtype

# Convert them to numeric  
df[['min_salary', 'max_salary', 'avg_salary']] = df[['min_salary', 'max_salary', 'avg_salary']].astype(np.float16)

In [73]:
df.head()
df.dtypes

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,hourly,employer_provided,min_salary,max_salary,avg_salary
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53.0,91.0,72.0
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,0,63.0,112.0,87.5
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,0,0,80.0,90.0,85.0
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",0,0,56.0,97.0,76.5
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86.0,143.0,114.5


Unnamed: 0             int64
Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Headquarters          object
Size                  object
Founded                int64
Type of ownership     object
Industry              object
Sector                object
Revenue               object
Competitors           object
hourly                 int64
employer_provided      int64
min_salary           float16
max_salary           float16
avg_salary           float16
dtype: object

In [74]:
#((df["Rating"] < 0) * 1).unique()
print("Total # of Companies not having Rating:")
((df["Rating"] < 0) * 1).sum() # there 11 Companies without rating Tagged: -1 

Total # of Companies not having Rating:


11

In [75]:
# COMPANY NAME parsing  
# Extract "Company Name" from Company Name and extract Ratings if  
df["company_name"] = df.apply(lambda x: x["Company Name"] if x["Rating"] < 0 else x["Company Name"].split('\n')[0], axis=1)
print("Total # of Companies:")
df["company_name"].value_counts()


Total # of Companies:


Takeda Pharmaceuticals                             14
MassMutual                                         14
Reynolds American                                  14
Software Engineering Institute                     11
Liberty Mutual Insurance                           10
PNNL                                               10
AstraZeneca                                         9
MITRE                                               8
Pfizer                                              7
Rochester Regional Health                           7
Fareportal                                          7
Advanced BioScience Laboratories                    7
Novartis                                            7
Numeric, LLC                                        7
Kronos Bio                                          6
The Church of Jesus Christ of Latter-day Saints     6
Tapjoy                                              6
Novetta                                             6
Esri                        

In [76]:
# STATE NAME parsing 
df["state_name"] = df["Location"].apply(lambda x: x.split(',')[1])
# df["state_name"].value_counts()
df["state_name"].unique()

# Find out if "State" is the same as the "Headquarter" 
df["same_state_as_hq"] = df.apply(lambda x: 1 if x["Location"] == x["Headquarters"] else 0, axis=1)
df["same_state_as_hq"].value_counts()


array([' NM', ' MD', ' FL', ' WA', ' NY', ' TX', ' CA', ' VA', ' MA',
       ' NJ', ' CO', ' IL', ' KY', ' OR', ' CT', ' MI', ' DC', ' OH',
       ' AL', ' MO', ' PA', ' GA', ' IN', ' LA', ' WI', ' NC', ' AZ',
       ' NE', ' MN', ' Los Angeles', ' UT', ' TN', ' DE', ' ID', ' RI',
       ' IA', ' SC', ' KS'], dtype=object)

1    414
0    328
Name: same_state_as_hq, dtype: int64

In [77]:
# AGE OF COMPANIES parsing, keeping -1 if Founded year is not given 
df["age"] = df["Founded"].apply(lambda x: x if x < 0 else (2020 - x))
df["age"].value_counts()
df["age"].dtypes

-1      50
 10     32
 12     31
 24     27
 14     24
 8      21
 9      19
 62     18
 18     18
 36     18
 13     18
 5      16
 7      15
 169    14
 145    14
 23     14
 239    14
 6      13
 55     12
 21     12
 3      12
 85     10
 20     10
 108    10
 17     10
 15     10
 107     9
 38      9
 39      9
 59      9
        ..
 33      2
 46      2
 69      2
 37      2
 45      2
 68      2
 66      2
 40      2
 58      2
 92      2
 91      1
 49      1
 75      1
 90      1
 170     1
 160     1
 208     1
 78      1
 142     1
 35      1
 137     1
 93      1
 41      1
 133     1
 131     1
 121     1
 118     1
 106     1
 103     1
 276     1
Name: age, Length: 102, dtype: int64

dtype('int64')

In [78]:
import re
# JOB DESCRIPTION parsing, extract relevant keywords
# Find out relevant SKILLS(sk) keyword in job descriptions
df["python_sk"] = df["Job Description"].apply(lambda x: 1 if 'python' in x.lower() else 0)
df["rstudio_sk"] = df["Job Description"].apply(lambda x: 1 if re.search(r'\br studio\b', x.lower()) or re.search(r'\br-studio\b', x.lower()) else 0)
df["spark_sk"] = df["Job Description"].apply(lambda x: 1 if re.search(r'\bspark\b', x.lower()) else 0)
df["aws_sk"] = df["Job Description"].apply(lambda x: 1 if re.search(r'\baws\b', x.lower()) else 0)
df["excel_sk"] = df["Job Description"].apply(lambda x: 1 if re.search(r'\bexcel\b', x.lower()) else 0)
df["sql_sk"] = df["Job Description"].apply(lambda x: 1 if re.search(r'\bsql\b', x.lower()) else 0)
df["tableau_sk"] = df["Job Description"].apply(lambda x: 1 if 'tableau' in x.lower() else 0)
df["nosql_sk"] = df["Job Description"].apply(lambda x: 1 if 'nosql' in x.lower() else 0)


# Find out relevant QUALIFICATIONS(ql) keyword in job descriptions
df["comp_sc_ql"] = df["Job Description"].apply(lambda x: 1 if 'computer science' in x.lower() else 0)
df["stat_ql"] = df["Job Description"].apply(lambda x: 1 if 'statistic' in x.lower() else 0)
df["phd_ql"] = df["Job Description"].apply(lambda x: 1 if re.search(r'\bphd\b', x.lower()) else 0)
df["master_ql"] = df["Job Description"].apply(lambda x: 1 if re.search(r'\bmaster\b', x.lower()) or re.search(r'\bmsc\b', x.lower()) or re.search(r'\bmsc.\b', x.lower()) else 0)
df["bsc_ql"] = df["Job Description"].apply(lambda x: 1 if 'bachelor' in x.lower() else 0)

In [79]:
# Example 
df["python_sk"].value_counts()

1    392
0    350
Name: python_sk, dtype: int64

In [80]:
# Save dataset having "Salary Estimate"
df.columns
df = df.drop(["Unnamed: 0"], axis=1)
# Save 
df.to_csv('dataset/gd_data_cleaned_with_salary_estimate.csv', index=False)

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_name', 'state_name', 'same_state_as_hq', 'age', 'python_sk',
       'rstudio_sk', 'spark_sk', 'aws_sk', 'excel_sk', 'sql_sk', 'tableau_sk',
       'nosql_sk', 'comp_sc_ql', 'stat_ql', 'phd_ql', 'master_ql', 'bsc_ql'],
      dtype='object')